## 1.  Import libraries :
Import two libraries for this assignment:
* pandas (for dataframe, included in Anaconda Python 3.7) 
* re (for regular expression, included in Anaconda Python 3.7) 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd
import re

## 2. Open file
* The First step, open the Group081.txt, transform information of each ID to one line, and write to Temp.txt
* The Second step, open the Group081.json, and write '{' at the start 

In [2]:
# input the file
text = open('Group081.txt','r')
# input the file
temp = open('Temp.txt', 'w')

# read each line in file
for line in text:
    #if line is not end with </us-patent-grant>\n(the pattern of each ID's information end), replace \n to ' '
    if line != '</us-patent-grant>\n':
        line = line.replace('\n',' ')
        temp.write(line)
    else:
        temp.write(line)

# close the Group081.txt and Temp.txt
text.close()
temp.close()

In [3]:
# input the file
jsonFile = open('Group081.json', 'w')

#write '{' at the start
jsonFile.write('{')

# close the Group081.json
jsonFile.close()

# input the file
jsonFile = open('Group081.json', 'a+')

## 3. Find the pattern
In this section, find the pattern of following elements:
* Grant_id: a unique ID for a patent grant consisting of alphanumeric characters.
* Patent_kind: a category to which the patent grant belongs.
* Patent_title: a title given by the inventor to the patent claim.
* Number_of_claims: an integer denoting the number of claims for a given grant.
* Citations_examiner_count: an integer denoting the number of citations made by the examiner for a given patent grant (0 if None)
* Citations_applicant_count: an integer denoting the number of citations made by the applicant for a given patent grant (0 if None)
* Inventors: a list of the patent inventors’ names ([NA] if the value is Null).
* Claims_text: a list of claim texts for the different patent claims ([NA] if the value is Null). 
* Abstract: the patent abstract text (‘NA’ if the value is Null)

In [4]:
# set the identify patterns for ID
patternID = re.compile(r'dtd-version=.*file="(.*?)-')

# set the identify patterns for Title
patternTitle = re.compile(r'<invention-title.*?>(.*)</invention-title>')

# set the identify patterns for Kind
patternKind = re.compile(r'appl-type=\"(.*?)\"')

# set the identify patterns for Number_of_claims
patternClaimNo = re.compile(r'<number-of-claims>(.*?)</number-of-claims>')

# set the identify patterns for Inventors
patternInventor = re.compile(r'<inventor.*?<last-name>(.*?)</last-name> <first-name>(.*?)</first-name>')

# set the identify patterns for citations_applicant_count
patternApplicant = re.compile(r'<category>(.*?applicant)</category>')

# set the identify patterns for citations_examiner_count
patternExaminer = re.compile(r'<category>(.*?examiner)</category>')

# set the identify patterns for claims_text
patternClaims = re.compile(r'<claim-text>(.*?)</claim-text>')

# set the identify patterns for Abstract
patternAbstract = re.compile(r'</us-bibliographic-data-grant> <abstract.*?> <p.*?>(.*?)</p>')

# set the identify patterns for Pubilished
patternPublished= re.compile(r'dtd-version=.*file=\".*?-.*?<kind>(\w\d)</kind>')

# define column_name variable as a list
column_name=['patent_title', 'kind', 'number_of_claims', 'inventors','citations_applicant_count','citations_examiner_count','claims_text','abstract']

# define idList, titleList, kindList, claimNoList, inventorList, countAppList, countExamList, claimsList, and abstractList variable as a list
idList=[]
titleList=[]
kindList=[]
claimNoList=[]
inventorList=[]
countAppList=[]
countExamList=[]
claimsList=[]
abstractList=[]
totalNumber=0

## 4. Find the total number 
In this section, find the total number of this file

In [5]:
# input the file
temp = open('Temp.txt', 'r')

# Read each line in file, and plus one to total_number
for line in temp:
    totalNumber+=1

# close the file
temp.close()

## 5. Parse Temp.txt File and Output the Group081.json
* In the first of this section, parse the temp by using the pattern we found in Step3.
* In the second of this section, write the information that we parse to Group081.json

In [6]:
# input the file
temp = open('Temp.txt', 'r')

# Read each line in file
for line in temp:
    
    # Parse ID in each line, and append to idList
    ID = patternID.search(line).group(1)
    idList.append(ID)
    
    # Parse title in each line, and append to titleList
    title = patternTitle.search(line).group(1)
    titleList.append(title)
    
    # Parse title in each line
    matchkind = patternKind.search(line).group(1)
    matchpublish= patternPublished.search(line).group(1)
    # If kind is 'utility' and publishkind is B2, kind is 'Utility Patent Grant (with a published application) issued on or after January 2, 2001.'
    if matchkind == 'utility':
        if matchpublish == 'B2':
            kind='Utility Patent Grant (with a published application) issued on or after January 2, 2001.'
        # If kind is 'utility' and publishkind is B1, kind is 'Utility Patent Grant (no published application) issued on or after January 2, 2001.'
        elif matchpublish == 'B1':
            kind='Utility Patent Grant (no published application) issued on or after January 2, 2001.'
    # If kind is 'plant' and publishkind is P3, kind is 'Plant Patent Grant (with a published application) issued on or after January 2, 2001'
    elif matchkind == 'plant':
        if matchpublish == 'P3':
            kind='Plant Patent Grant (with a published application) issued on or after January 2, 2001'
        # If kind is 'plant' and publishkind is P2, kind is 'Plant Patent Grant (no published application) issued on or after January 2, 2001'
        elif matchpublish =='P2':
            kind ='Plant Patent Grant (no published application) issued on or after January 2, 2001'
    else:
        kind= str(matchkind).capitalize() +' Patent'   
    #append kind to kindList    
    kindList.append(kind)
    
    # Parse Number_of_claims in each line, and append to claimNoList
    claimNo = patternClaimNo.search(line).group(1)
    claimNoList.append(claimNo)
    
    # Parse Inventors in each line, and append to inventorList
    matchInventor = patternInventor.findall(line)
    for i in range(len(matchInventor)):
        matchInventor[i] = list(matchInventor[i])
        matchInventor[i][0], matchInventor[i][1] = matchInventor[i][1], matchInventor[i][0]
        matchInventor[i] = " ".join(matchInventor[i])
    inventor = ",".join(matchInventor)
    inventor = '[' + inventor + ']'
    inventorList.append(inventor)
 
    # Parse citations_applicant_count in each line, and append to countAppList
    matchApplicant = patternApplicant.findall(line)
    countApplicant = str(len(matchApplicant))
    countAppList.append(countApplicant)
    
    # Parse citations_examiner_count in each line, and append to countAppList
    matchExaminer = patternExaminer.findall(line)
    countExaminer = str(len(matchExaminer))
    countExamList.append(countExaminer)
    
    # Parse claims_text in each line, remove all the tag in claims_text, and append to claimList
    matchClaims = patternClaims.findall(line)
    claimsWithTag = " ".join(matchClaims)
    claims = re.sub(r'<.*?>','',claimsWithTag)
    claims = '[' + claims + ']'
    claimsList.append(claims)
    
    
    # Parse Abstract in each line, append to countAppList
    matchAbstract = patternAbstract.search(line)
    #if there is nothing in abstract, abstract is 'NA'
    if matchAbstract:
        abstract = matchAbstract.group(1)
    else:
        abstract = 'NA'
    abstractList.append(abstract)

    #write the information to Group081.json
    if len(idList) == totalNumber:
        jsonFile.write('"' + ID + '":{"patent_title":"' + title + '","kind":"' + kind + '","number_of_claims":' + claimNo + ',"inventors":"' + inventor + '","citations_applicant_count":' + countApplicant + ',"citations_examiner_count":' + countExaminer + ',"claims_text":"' + claims + '","abstract":"' + abstract + '"}')
    else:
        jsonFile.write('"' + ID + '":{"patent_title":"' + title + '","kind":"' + kind + '","number_of_claims":' + claimNo + ',"inventors":"' + inventor + '","citations_applicant_count":' + countApplicant + ',"citations_examiner_count":' + countExaminer + ',"claims_text":"' + claims + '","abstract":"' + abstract + '"},')

        
# close the Group081.json
jsonFile.close()
# input the file
jsonFile = open('Group081.json','a')
# write the '}' to the end of file
jsonFile.write('}')
# close the Group081.json
jsonFile.close()
# close the file
temp.close()


## 6. Output CSV file
In this section, use pandas to create the dataframe, and output the CSV file

In [7]:
# define the csvdata varible as a dict
csvdata={'grant_id':idList,
      'patent_title': titleList,
      'kind': kindList,
      'number_of_claims': claimNoList,
      'inventors': inventorList,
      'citations_applicant_count': countAppList,
      'citations_examiner_count': countExamList,
      'claims_text': claimsList,
      'abstract': abstractList}

# use DataFrame() to transfer the csvdata to csvdataframe
csvdataframe=pd.DataFrame(csvdata)
# set the index of the csvdataframe
df1=csvdataframe.set_index('grant_id')
#output the csv file
df1.to_csv("Group081.csv")    